In [42]:
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._

val transactionsRDD = sc.textFile("test2.csv")
val schemaString = "TransID CustID TransTotal TransNumItems TransDesc"

val fields = schemaString.split(" ").
    map(fieldName => StructField(fieldName, StringType, nullable = true))
val schema = StructType(fields)

val rowRDD = transactionsRDD.
    map(_.split(",")).
    map(attributes => Row(attributes(0), attributes(1), attributes(2), attributes(3), attributes(4).trim))
val transactionsDF = spark.createDataFrame(rowRDD, schema)
transactionsDF.createOrReplaceTempView("transactions")


import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
transactionsRDD: org.apache.spark.rdd.RDD[String] = test2.csv MapPartitionsRDD[128] at textFile at <console>:43
schemaString: String = TransID CustID TransTotal TransNumItems TransDesc
fields: Array[org.apache.spark.sql.types.StructField] = Array(StructField(TransID,StringType,true), StructField(CustID,StringType,true), StructField(TransTotal,StringType,true), StructField(TransNumItems,StringType,true), StructField(TransDesc,StringType,true))
schema: org.apache.spark.sql.types.StructType = StructType(StructField(TransID,StringType,true), StructField(CustID,StringType,true), StructField(TransTotal,StringType,true), StructField(TransNumItems,StringType,true), StructField(TransDesc,StringType,true))
rowRDD: org.apache.spar...

## T1:	Filter	out	(drop)	the	transactions	from	T whose	total	amount	is	less	than	$200

In [43]:
val T1 = spark.sql("""
    SELECT *
    FROM transactions
    WHERE TransTotal > 200
    """)
transactionsDF.createOrReplaceTempView("T1")

T1: org.apache.spark.sql.DataFrame = [TransID: string, CustID: string ... 3 more fields]


In [44]:
T1.show()

+-------+------+----------+-------------+--------------------+
|TransID|CustID|TransTotal|TransNumItems|           TransDesc|
+-------+------+----------+-------------+--------------------+
|      1| 13931| 947.27325|           10|7tk17k7r14ixnkgbw9xc|
|      3| 10311|  563.8975|           10|9yc74al32gyx4hmpu...|
|      4| 14630|  737.5072|            4|bdnglgyulcln7zl9i...|
|      5| 38528| 520.28046|            3|9nvr6vb2ouppqqv8l...|
|      6|  4505|  694.3501|            6|j6eoz2r5swlpjpxkq...|
|      7| 12466|  283.8117|            8|wrwfqu49uakdxqq2c...|
|      8| 16572|  331.2615|            6|7bq937yzswhhlejos...|
|      9| 18696|  380.0146|            1|v77phaie3detj3tm5...|
|     10| 11084|  781.5895|            1|f5sj0wmi9pmmmkc5w...|
|     11|  8115|  958.5794|            2|igxkh8nzn7iralfn4...|
|     13| 36239|  302.1748|            9|iyo7s15aib3ll9ug3...|
|     14| 45097| 206.60675|            5|dlcq63uctjfrtyjvc...|
|     15| 23428|  907.9976|            7|hura8k4tv7w8u6

## T2:	Over	T1,	group	the	transactions	by	the	Number	of	Items	it	has,	and	for	each	group	calculate	the	sum	of	total	amounts,	the	average	of	total	amounts,	the	min	and	the	max	of	the	total	amounts.	

In [45]:
var T2 = spark.sql("""
    SELECT TransNumItems, sum(TransTotal) as sum, avg(TransTotal) as avg, min(TransTotal) as min, max(TransTotal) as max
    FROM T1
    GROUP BY TransNumItems
    """)
transactionsDF.createOrReplaceTempView("T2")

T2: org.apache.spark.sql.DataFrame = [TransNumItems: string, sum: double ... 3 more fields]


## 3) Report	back	T2	to	the	client	side

In [46]:
T2.show()

+-------------+------------------+------------------+---------+---------+
|TransNumItems|               sum|               avg|      min|      max|
+-------------+------------------+------------------+---------+---------+
|            7| 503184.9979949998| 507.7547911150351|10.006294|999.63135|
|            3| 513023.1376229998|510.97922074003964|10.366854| 999.9228|
|            8| 496979.5531750001|497.97550418336687|100.24613| 999.2891|
|            5| 500082.7098709998| 496.1137994751982| 10.64183| 998.9041|
|            6| 515128.0622620001|508.01584049506914|10.054651| 999.7996|
|            9| 529327.5989350002|503.16311685836524|10.484435| 996.2176|
|            1|516024.74242849986| 500.0239752214146|100.03025| 997.6773|
|           10| 450016.9132989999| 498.9101034356983|100.17128| 998.7193|
|            4| 503476.3507269997| 506.5154433873236|100.71896|998.41284|
|            2| 512846.8069620001|510.29533031044787| 100.8176|999.82263|
+-------------+------------------+----

## 4) T3:	Over	T1,	group	the	transactions	by	customer	ID,	and	for	each	group	report	the	customer	ID,	and	the	transactions’	count.	

In [66]:
var T3 = spark.sql("""
    SELECT CustID, count(*) as TransCount
    FROM T1
    GROUP BY CustID
    """)
transactionsDF.createOrReplaceTempView("T3")
T3.show()

+------+----------+
|CustID|TransCount|
+------+----------+
| 18726|         1|
| 20569|         1|
| 48402|         1|
| 19095|         1|
| 32275|         1|
|  7273|         1|
| 18556|         1|
|  3414|         2|
| 31432|         2|
| 31713|         2|
|  3210|         1|
|  4821|         1|
| 44446|         1|
| 29549|         2|
| 21452|         1|
| 14899|         1|
| 45157|         2|
| 22121|         1|
| 21889|         1|
| 39659|         1|
+------+----------+
only showing top 20 rows



T3: org.apache.spark.sql.DataFrame = [CustID: string, TransCount: bigint]


## 5) T4:	Filter	out	(drop)	the	transactions	from	T whose	total	amount	is	less	than $600

In [48]:
var T4 = spark.sql("""
    SELECT *
    FROM transactions
    WHERE TransTotal < 600
    """)
transactionsDF.createOrReplaceTempView("T4")

T4: org.apache.spark.sql.DataFrame = [TransID: string, CustID: string ... 3 more fields]


## 6) T5:	Over	T4,	group	the	transactions	by	customer	ID,	and	for	each	group	report	the	customer	ID,	and	the	transactions’	count.	

In [71]:
var T5 = spark.sql("""
    SELECT CustID, count(*) as TransCount
    FROM T4
    GROUP BY CustID
    """)
transactionsDF.createOrReplaceTempView("T5")
T5.show()


+------+----------+
|CustID|TransCount|
+------+----------+
| 18726|         1|
| 20569|         1|
| 48402|         1|
| 19095|         1|
| 32275|         1|
|  7273|         1|
| 18556|         1|
|  3414|         2|
| 31432|         2|
| 31713|         2|
|  3210|         1|
|  4821|         1|
| 44446|         1|
| 29549|         2|
| 21452|         1|
| 14899|         1|
| 45157|         2|
| 22121|         1|
| 21889|         1|
| 39659|         1|
+------+----------+
only showing top 20 rows



T5: org.apache.spark.sql.DataFrame = [CustID: string, TransCount: bigint]


## 7) T6:	Select	the	customer	IDs	whose		T5.count	*	5 <	T3.count

In [53]:
var T6 = spark.sql("""
    SELECT T5.CustID
    FROM T5
    JOIN T3
    ON T5.CustID = T3.CustID
    WHERE T5.TransCount * 5 < T3.TransCount
    GROUP BY T5.CustID
    """)
transactionsDF.createOrReplaceTempView("T6")
T6.show()

org.apache.spark.sql.AnalysisException:  cannot resolve '`T5.TransCount`' given input columns: [t5.TransID, t5.TransDesc, t3.TransDesc, t3.TransTotal, t5.TransTotal, t3.TransID, t5.TransNumItems, t3.TransNumItems, t5.CustID, t3.CustID]; line 6 pos 10;